In [24]:
from io import StringIO
import requests
import os
import pandas as pd


url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=60min&apikey={os.getenv("ALPHAVANTAGE_API_KEY")}&datatype=csv'
response = requests.get(url)
data = StringIO(response.text)
df = pd.read_csv(data)
display(df)


,timestamp,open,high,low,close,volume
0,2024-09-24 19:00:00,220.97,221.12,220.70,221.11,771533
1,2024-09-24 18:00:00,220.96,221.10,220.91,221.10,771453
2,2024-09-24 17:00:00,220.95,220.97,220.70,220.95,163
3,2024-09-24 16:00:00,221.01,221.25,220.00,220.95,2378426
4,2024-09-24 15:00:00,220.09,221.16,220.01,221.01,445715
...,...,...,...,...,...,...
95,2024-09-17 04:00:00,217.15,218.89,217.15,217.49,164
96,2024-09-16 19:00:00,217.16,217.50,217.11,217.30,641533
97,2024-09-16 18:00:00,217.11,217.50,217.11,217.16,641591
98,2024-09-16 17:00:00,217.37,217.40,217.11,217.12,1975


In [25]:
import numpy as np
 
class MinMaxScaler:
    def __init__(self, feature_range=(0, 1)):
        self.min_ = None  # Minimum of each feature in the training set
        self.max_ = None  # Maximum of each feature in the training set
        self.data_min_ = None  # Min value found during fit
        self.data_max_ = None  # Max value found during fit
        self.scale_ = None  # Scale factor for each feature
        self.feature_range = feature_range  # Desired output range (default: 0 to 1)

    def fit(self, X):
        """
        Compute the minimum and maximum to be used for later scaling.
        """
        self.data_min_ = np.min(X, axis=0)
        self.data_max_ = np.max(X, axis=0)

        # Range of the original data
        data_range = self.data_max_ - self.data_min_

        # Desired output range
        feature_range_min, feature_range_max = self.feature_range

        # Scale factor is the ratio of the desired range to the original range
        self.scale_ = (feature_range_max - feature_range_min) / data_range
        self.min_ = feature_range_min - self.data_min_ * self.scale_

        return self

    def transform(self, X):
        """
        Scale the data according to the previously computed min_ and scale_.
        """
        return X * self.scale_ + self.min_

    def fit_transform(self, X):
        """
        Fit to data, then transform it.
        """
        return self.fit(X).transform(X)

# data = data.readlines()
close_prices = df['close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)


print(close_prices_scaled)


[[1.        ]
 [0.99897751]
 [0.98364008]
 [0.98364008]
 [0.98977505]
 [0.89672802]
 [0.92586912]
 [0.95245399]
 [0.9202454 ]
 [0.98108384]
 [0.78271984]
 [0.90490798]
 [0.95092025]
 [0.95194274]
 [0.98364008]
 [0.98466258]
 [0.93456033]
 [0.93660532]
 [0.91820041]
 [0.92740286]
 [0.93251534]
 [0.90388548]
 [0.88957055]
 [0.8502045 ]
 [0.7597137 ]
 [0.80214724]
 [0.61554192]
 [0.58997955]
 [0.64621677]
 [0.65132924]
 [0.6206544 ]
 [0.5807771 ]
 [0.64785276]
 [0.63599182]
 [0.62218814]
 [0.6497955 ]
 [0.64621677]
 [0.60429448]
 [0.55112474]
 [0.4846728 ]
 [0.48568507]
 [0.39059305]
 [0.33844581]
 [0.26482618]
 [0.22290389]
 [0.25562372]
 [0.26584867]
 [0.22801636]
 [0.22188139]
 [0.25255624]
 [0.22188139]
 [0.24130879]
 [0.26175869]
 [0.20961145]
 [0.20655419]
 [0.15439673]
 [0.        ]
 [0.09713701]
 [0.34355828]
 [0.50306748]
 [0.57770961]
 [0.54907975]
 [0.54907975]
 [0.51840491]
 [0.44478528]
 [0.38241309]
 [0.37832311]
 [0.42331288]
 [0.36400818]
 [0.42944785]
 [0.37218814]
 [0.38

In [27]:
import tensorflow as tf

def create_lstm_data(data, time_steps=1):
 x, y = [], []
 for i in range(len(data) - time_steps):
  x.append(data[i:(i + time_steps), 0])
  y.append(data[i + time_steps, 0])
 return np.array(x), np.array(y)

time_steps = 100
x, y = create_lstm_data(close_prices_scaled, time_steps)
x = np.reshape(x, (x.shape[0], x.shape[1], 1))

model = tf.Sequential()
model.add(tf.LSTM(units=50, return_sequences=True, input_shape=(x.shape[1], 1)))
model.add(tf.LSTM(units=50))
model.add(tf.Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

model.fit(x, y, epochs=50, batch_size=32, validation_split=0.2)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject